For the batch scoring, we will persist the values in a new global persistent Databricks table. In production data workloads, you may save the scored data to Blob Storage, Azure Cosmos DB, or other serving layer. Another implementation detail we are skipping for the lab is processing only new files. This can be accomplished by creating a widget in the notebook that accepts a path parameter that is passed in from Azure Data Factory.

In [2]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, Bucketizer
from pyspark.sql.functions import array, col, lit
from pyspark.sql.types import *

Replace STORAGE-ACCOUNT-NAME with the name of your storage account. You can find this in the Azure portal by locating the storage account that you created in the lab setup, within your resource group. The container name is set to the default used for this lab. If yours is different, update the containerName variable accordingly.

In [4]:
accountName = "mlbricksdatastore"
containerName = "sparkcontainer"

In [5]:
# mounted the storage account so no need to do it again
#dbutils.fs.mount(
 #source = "wasbs://sparkcontainer@mlbricksdatastore.blob.core.windows.net/",
  # mount_point = "/mnt/mcw/",
   #extra_configs = {"fs.azure.sas.source.mlbrickstore.blob.core.windows.net": "?sv=sv=2018-03-28&ss=b&srt=sco&sp=rwdlac&se=2021-01-07T08:37:52Z&st=2019-01-07T00:37:52Z&spr=https&sig=3vforJjVn840tqhhydmu9XNdx8l6KC%2B2LJbT0zCEbHw%3D"})

--------------------------------------------------------------------------- 
 ExecutionError Traceback (most recent call last)
 <command-10036749142644> in <module> () 
 3 source = "wasbs://sparkcontainer@mlbricksdatastore.blob.core.windows.net/" , 
 4 mount_point = "/mnt/mcw/" , 
 ----> 5 extra_configs = {"fs.azure.sas.source.mlbrickstore.blob.core.windows.net": "?sv=sv=2018-03-28&ss=b&srt=sco&sp=rwdlac&se=2021-01-07T08:37:52Z&st=2019-01-07T00:37:52Z&spr=https&sig=3vforJjVn840tqhhydmu9XNdx8l6KC%2B2LJbT0zCEbHw%3D"})
 
 /local_disk0/tmp/1549159416164-0/dbutils.py in f_with_exception_handling (*args, **kwargs) 
 278 exc . __context__ = None 
 279 exc . __cause__ = None 
 --> 280 raise exc 
 281 return f_with_exception_handling 
 282 

 ExecutionError : An error occurred while calling o262.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: Could not retrieve either the Azure storage account access key through option key fs.azure.account.key.mlbricksdatastore.blob.core.windows.net or the SAS token for the Azure container through option key fs.azure.sas.sparkcontainer.mlbricksdatastore.blob.core.windows.net.; nested exception is: 
	java.lang.IllegalArgumentException: Could not retrieve either the Azure storage account access key through option key fs.azure.account.key.mlbricksdatastore.blob.core.windows.net or the SAS token for the Azure container through option key fs.azure.sas.sparkcontainer.mlbricksdatastore.blob.core.windows.net.
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:100)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:54)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.mount(DBUtilsCore.scala:451)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: Could not retrieve either the Azure storage account access key through option key fs.azure.account.key.mlbricksdatastore.blob.core.windows.net or the SAS token for the Azure container through option key fs.azure.sas.sparkcontainer.mlbricksdatastore.blob.core.windows.net.
	at com.databricks.backend.daemon.data.server.backend.FileSystemBackendManager.$anonfun$getAzureStorageCredentials$4(FileSystemBackendManager.scala:419)
	at scala.Option.getOrElse(Option.scala:138)
	at com.databricks.backend.daemon.data.server.backend.FileSystemBackendManager.getAzureStorageCredentials(FileSystemBackendManager.scala:415)
	at com.databricks.backend.daemon.data.server.backend.FileSystemBackendManager.resolveCredentials(FileSystemBackendManager.scala:182)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:43)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:83)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:82)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:82)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:272)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:252)
	at com.datab

Define the schema for the CSV files

In [7]:
data_schema = StructType([
        StructField('OriginAirportCode',StringType()),
        StructField('Month', IntegerType()),
        StructField('DayofMonth', IntegerType()),
        StructField('CRSDepHour', IntegerType()),
        StructField('DayOfWeek', IntegerType()),
        StructField('Carrier', StringType()),
        StructField('DestAirportCode', StringType()),
        StructField('DepDel15', IntegerType()),
        StructField('WindSpeed', DoubleType()),
        StructField('SeaLevelPressure', DoubleType()),  
        StructField('HourlyPrecip', DoubleType())])

Create a new DataFrame from the CSV files, applying the schema

In [9]:
dfDelays = spark.read.csv("wasbs://" + containerName + "@" + accountName + ".blob.core.windows.net/FlightsAndWeather/*/*/FlightsAndWeather.csv",
                    schema=data_schema,
                    sep=",",
                    header=True)

Load the trained machine learning model you created earlier in the lab

In [11]:
# Load the saved pipeline model
model = PipelineModel.load("/flightDelayModel")

Make a prediction against the loaded data set

In [13]:
# Make a prediction against the dataset
prediction = model.transform(dfDelays)

Save the scored data into a new global table called **scoredflights**

In [15]:
prediction.write.mode("overwrite").saveAsTable("scoredflights")